Для тестирования результатов в chromadb были использованы не только большое количество разных моделей, но и разные подходы к данным, которые мы загружали в базу данных.
Ниже представлена обработка двух таблиц:
1. Финальная таблица, которую мы использовали с дополнительной колонкой review_clean в разных вариациях без прилагательных и глаголов.
2. Таблица с фичами, или ключевыми словами, которые были найдены в отзывах.

К сожалению обработанные отзывы без лишних символов, в инфинитиве, прилагательных и глаголов и их вариации, а также фичи показали себя хуже в поиске и даже в простых запросах выдавали очевидно неправильные результаты.

# Загрузка файлов

In [14]:
import pandas as pd

df = pd.read_csv(
    r"review_flamp_yandex.csv",
    sep=",",
)
display(df.head(2))
dff = pd.read_csv(
    r"features.csv",
    sep=",",
)
display(dff.head(2))

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp


In [15]:
# замены пустых значений на пустую строку
df["review"] = df["review"].fillna("", inplace=False)
dff["reviews"] = dff["reviews"].fillna("", inplace=False)
dff["features"] = dff["features"].fillna("", inplace=False)

# Обработка

In [16]:
# функция обработки необходимых колонок. Принимает на вход датафреймы и функцию для обработки
def process_column(input_df, input_dff, func):
    df = input_df.copy()
    dff = input_dff.copy()

    df["review_clean"] = df["review_clean"].apply(func)
    dff["reviews_clean"] = dff["reviews_clean"].apply(func)
    dff["features_clean"] = dff["features_clean"].apply(func)
    display(df.head(2))
    display(dff.head(2))

    return df, dff

In [17]:
# инициализация датафреймов
initial_df = df.copy()
initial_dff = dff.copy()

df = initial_df.copy()
dff = initial_dff.copy()

df["review_clean"] = df["review"]
dff["reviews_clean"] = dff["reviews"]
dff["features_clean"] = dff["features"]

## 1. Приведение к нижнему регистру

In [18]:
# Приведение к нижнему регистру
def to_lower(text):
    return text.lower()


df_1, dff_1 = process_column(df, dff, to_lower)

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,"в принципе, нормальный ресторанчик при отеле. ..."
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,"есть такой ресторан на юге москвы, по названию..."


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source,reviews_clean,features_clean
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к..."
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело..."


## 2. удаление лишних символов

In [ ]:
# удаление лишних символов
import re


def clean_text(text):
    return re.sub(r"[^\w\s\n]", " ", text)


df_2, dff_2 = process_column(df_1, dff_1, clean_text)

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,в принципе нормальный ресторанчик при отеле по...
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,есть такой ресторан на юге москвы по названию ...


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source,reviews_clean,features_clean
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp,в принципе нормальный ресторанчик при отеле по...,бизнесланч биточки вкусная еда гречотто кофе л...
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp,есть такой ресторан на юге москвы по названию ...,бизнесланчи вкусная еда вкусный стейк деловой ...


## 3. удаление пунктуации

In [ ]:
# удаление пунктуации
import string


def remove_punctuation(text):
    # Удаляем знаки пунктуации
    translator = str.maketrans(string.punctuation, " " * len(string.punctuation))
    return text.translate(translator).strip()


df_3, dff_3 = process_column(df_2, dff_2, remove_punctuation)

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,в принципе нормальный ресторанчик при отеле по...
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,есть такой ресторан на юге москвы по названию ...


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source,reviews_clean,features_clean
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp,в принципе нормальный ресторанчик при отеле по...,бизнесланч биточки вкусная еда гречотто кофе л...
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp,есть такой ресторан на юге москвы по названию ...,бизнесланчи вкусная еда вкусный стейк деловой ...


## 4. лемматизация

In [22]:
# Лемматизация
import spacy

# Загружаем модель для русского языка
nlp = spacy.load("ru_core_news_sm")


def lemmatize(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)


df_4, dff_4 = process_column(df_3, dff_3, lemmatize)

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,в принцип нормальный ресторанчик при отель пос...
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,быть такой ресторан на юг москва по название в...


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source,reviews_clean,features_clean
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp,в принцип нормальный ресторанчик при отель пос...,бизнесланч биточки вкусный еда гречотто кофе л...
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp,быть такой ресторан на юг москва по название в...,бизнесланчи вкусный еда вкусный стейк деловой ...


## 5. удаление стоп-слов

In [23]:
# Удаляем стоп-слова
no_stopwords_reviews = []


def remove_stopwords(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    # Удаляем только те токены, которые не являются стоп-словами
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered_tokens)


df_5, dff_5 = process_column(df_4, dff_4, remove_stopwords)

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,принцип нормальный ресторанчик отель выезд ход...
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,ресторан юг москва название окунуться франц...


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source,reviews_clean,features_clean
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp,принцип нормальный ресторанчик отель выезд ход...,бизнесланч биточки вкусный еда гречотто кофе л...
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp,ресторан юг москва название окунуться франц...,бизнесланчи вкусный еда вкусный стейк деловой ...


## 6. удаление общих слов

In [ ]:
# удаление общих слов
unwanted_words = [
    "съедобный",
    "простой",
    "нейтральный",
    "обычный",
    "комфортный",
    "привычный",
    "тёплый",
    "спокойный",
    "понятный",
    "общий",
    "быть",
    "такой",
    "какой",
    "еда",
    "вкусный",
    "нормальный",
]
unwanted_words = set(unwanted_words)


def remove_unwanted_words(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    # Удаляем только те токены, леммы которых присутствуют в unwanted_adjectives
    filtered_tokens = [
        token.text for token in doc if token.lemma_ not in unwanted_words
    ]
    return " ".join(filtered_tokens)


"""
Не было использовано, так  было решено, что удаляет полезную информации. Вместо этого было решено сделать дополнительные колонки, где вообще не будет прилагательных или глаголов.

df_6, dff_6 = process_column(df_5, dff_5, remove_unwanted_words)
"""

## 7. удаление глаголов

In [37]:
def remove_verbs(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    # Удаляем все токены, у которых POS-тег VERB
    filtered_tokens = [token.text for token in doc if token.pos_ != "VERB"]
    filtered_text = " ".join(filtered_tokens)
    # Очистка лишних пробелов и знаков препинания
    filtered_text = re.sub(r"\s+,", ",", filtered_text)
    filtered_text = re.sub(r",\s+", ", ", filtered_text)
    filtered_text = re.sub(r"\s+\.", ".", filtered_text)
    return filtered_text

In [38]:
df_5["review_clean_no_verbs"] = df_5["review_clean"].apply(remove_verbs)
dff_5["reviews_clean_no_verbs"] = dff_5["reviews_clean"].apply(remove_verbs)
dff_5["features_clean_no_verbs"] = dff_5["features_clean"].apply(remove_verbs)

## 8. удаление всех прилагательных - только для features

In [39]:
def remove_adjectives(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    # Удаляем все токены, у которых POS-тег ADJ
    filtered_tokens = [token.text for token in doc if token.pos_ != "ADJ"]
    filtered_text = " ".join(filtered_tokens)
    # Очистка лишних пробелов и знаков препинания
    filtered_text = re.sub(r"\s+,", ",", filtered_text)
    filtered_text = re.sub(r",\s+", ", ", filtered_text)
    filtered_text = re.sub(r"\s+\.", ".", filtered_text)
    return filtered_text

In [41]:
df_5["review_clean_no_adj"] = df_5["review_clean"].apply(remove_adjectives)
dff_5["reviews_clean_no_adj"] = dff_5["reviews_clean"].apply(remove_adjectives)
dff_5["features_clean_no_adj"] = dff_5["features_clean"].apply(remove_adjectives)

## удаление и прилагательных и глаголов

In [44]:
df_5["review_clean_no_verbs_no_adj"] = df_5["review_clean_no_verbs"].apply(
    remove_adjectives
)
dff_5["reviews_clean_no_verbs_no_adj"] = dff_5["reviews_clean_no_verbs"].apply(
    remove_adjectives
)
dff_5["features_clean_no_verbs_no_adj"] = dff_5["features_clean_no_verbs"].apply(
    remove_adjectives
)

In [46]:
df_5.to_csv("review_flamp_yandex_v7.csv", index=False)
dff_5.to_csv("review_flamp_yandex_v7_keywords.csv", index=False)

In [47]:
display(df_5.head(2))
display(dff_5.head(2))

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_clean,review_clean_no_verbs,review_clean_no_adj,review_clean_no_verbs_no_adj
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp,принцип нормальный ресторанчик отель выезд ход...,принцип нормальный ресторанчик отель выезд сюд...,принцип ресторанчик отель выезд ходить сюда би...,принцип ресторанчик отель выезд сюда бизнеслан...
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp,ресторан юг москва название окунуться франц...,ресторан юг москва название французский по...,ресторан юг москва название окунуться поли...,ресторан юг москва название полинезия бер...


,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,...,rating,source,reviews_clean,features_clean,reviews_clean_no_verbs,features_clean_no_verbs,reviews_clean_no_adj,features_clean_no_adj,reviews_clean_no_verbs_no_adj,features_clean_no_verbs_no_adj
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,...,5.0,flamp,принцип нормальный ресторанчик отель выезд ход...,бизнесланч биточки вкусный еда гречотто кофе л...,принцип нормальный ресторанчик отель выезд сюд...,бизнесланч биточки вкусный еда гречотто кофе л...,принцип ресторанчик отель выезд ходить сюда би...,бизнесланч биточки еда гречотто кофе латте цен...,принцип ресторанчик отель выезд сюда бизнеслан...,бизнесланч биточки еда гречотто кофе латте цен...
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,...,5.0,flamp,ресторан юг москва название окунуться франц...,бизнесланчи вкусный еда вкусный стейк деловой ...,ресторан юг москва название французский по...,бизнесланчи вкусный еда вкусный стейк деловой ...,ресторан юг москва название окунуться поли...,еда стейк обед стиль мастерклассы доверие кухн...,ресторан юг москва название полинезия бер...,еда стейк обед стиль мастерклассы доверие кухн...


In [63]:
df_5["len"] = df_5["review"].apply(len)
dff_5["len"] = dff_5["features_clean_no_verbs_no_adj"].apply(len)

grouped_stats = df_5.groupby("source")["len"].describe()
display(grouped_stats)
grouped_stats = dff_5.groupby("source")["len"].describe()
display(grouped_stats)

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
flamp,23127.0,421.914688,452.478034,82.0,175.0,269.0,484.0,7997.0
yandex,24707.0,294.361112,270.476964,6.0,142.0,211.0,354.0,4793.0


,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
flamp,2550.0,130.760000,134.525377,0.0,50.0,97.0,174.0,1575.0
yandex,7211.0,84.261406,64.774430,0.0,39.0,65.0,113.0,1028.0


# оценка результатов

In [59]:
df_6 = df_5.copy()
dff_6 = dff_5.copy()

df_6 = df_6[(df_6["len"] > 7)]
df_6.dropna(subset=["review_clean_no_verbs_no_adj"], inplace=True)
dff_6 = dff_6[(dff_6["len"] > 7)]
dff_6.dropna(subset=["reviews_clean_no_verbs_no_adj"], inplace=True)
dff_6.dropna(subset=["features_clean_no_verbs_no_adj"], inplace=True)

In [60]:
grouped_stats = df_6.groupby("source")["len"].describe()
display(grouped_stats)
grouped_stats = dff_6.groupby("source")["len"].describe()
display(grouped_stats)

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
flamp,23123.0,155.723392,173.191126,8.0,63.0,101.0,180.0,4721.0
yandex,24565.0,114.388683,103.953868,8.0,54.0,84.0,139.0,2183.0


,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
flamp,2540.0,131.261024,134.552189,8.0,50.0,98.0,174.0,1575.0
yandex,7105.0,85.464180,64.496501,8.0,39.0,66.0,114.0,1028.0


In [61]:
df_6.to_csv("review_flamp_yandex_v7.csv", index=False)
dff_6.to_csv("review_flamp_yandex_v7_keywords.csv", index=False)

# Сплит

После нескольких тестирований мы увидели, что коротки отзывы появляются в результатах чаще, чем длинные. Поэтому решили убрать отзывы, где меньше чем 100 символов, а отзывы, где больше 500 символов - разбить на несколько частей.

In [81]:
def split_review(review: str, min_len=100, max_len=500):
    # Удаляем строки короче 100 символов
    if len(review) < min_len:
        return []

    # Если длина уже в пределах [100;500], просто возвращаем в списке
    if len(review) <= max_len:
        return [review]

    # Иначе разбиваем на части
    parts = []
    start = 0
    n = len(review)

    while start < n:
        # Определяем максимально допустимый конец текущего куска
        end = min(start + max_len, n)

        # Если мы уже близко к концу, и оставшийся текст короче 100 символов, то проверяем,
        # стоит ли вообще добавлять этот кусок
        if n - start < min_len:
            # Оставшийся текст слишком короткий, пропускаем
            break

        # Если текущий блок итак удовлетворяет условию [100;500], проверим, не надо ли его корректировать по пробелу
        # Попытаемся найти пробел, чтобы не обрывать слово
        # Если end == n (конец строки), то ничего искать не нужно
        if end < n:
            # Найдем ближайший пробел с конца до текущей максимальной границы
            space_pos = review.rfind(" ", start, end)
            if space_pos == -1:
                # Не нашли пробел, придётся отрезать на max_len символах.
                # Но если кусок меньше 100 символов, это плохо, тогда придётся сдвигать границу вперёд пока не найдём пробел или
                # достичь min_len.
                # Однако если нет пробела до max_len, просто разрежем по max_len (не очень красиво, но выбора нет)
                space_pos = end
            else:
                # Проверим длину найденного куска
                if space_pos - start < min_len:
                    # Если слишком коротко, попробуем найти пробел после space_pos, двигаясь вперёд
                    # чтобы был кусок не менее min_len.
                    forward_space = review.find(" ", space_pos + 1, end)
                    if forward_space == -1:
                        # Нет подходящего пробела дальше - придется взять max_len символов
                        space_pos = end
                    else:
                        # Если forward_space всё еще не дотягивает до min_len, будем искать дальше
                        # Но чтобы упростить, можно просто взять end, если не получается
                        # Это сложная логика, можно упростить, но для наглядности пусть так
                        if forward_space - start < min_len:
                            space_pos = end
                        else:
                            space_pos = forward_space
        else:
            # Здесь end == n, то есть мы на последнем куске
            space_pos = end

        current_part = review[start:space_pos].strip()

        # Если текущий кусок всё-таки меньше 100, то нужно либо расширить его, либо отказаться
        if len(current_part) < min_len:
            # Попытка расширить:
            # Если пространство для расширения есть (end < n), расширим до max_len или конца строки
            # Но мы уже пытались. В случае если расширить нельзя - пропускаем остаток.
            # Здесь для упрощения просто выйдем из цикла.
            break

        parts.append(current_part)
        start = space_pos
        # Пропускаем возможные лишние пробелы
        while start < n and review[start] == " ":
            start += 1

    return parts

In [82]:
df_7 = df_6.copy()
df_7["split_reviews"] = df_7["review"].apply(split_review)

In [ ]:
df_7 = df_7[df_7["split_reviews"].apply(lambda x: len(x) > 0)]
df_7 = df_7.explode("split_reviews").reset_index(drop=True)

In [88]:
df_7.to_csv("v1.csv", index=False)

In [ ]:
df_7 = df_7[(df_7["rating"] > 3) & (df_7["rating_decimal"] > 3)]

## dff_7

Тоже самое разбиение по количесту символов было сделано для таблицы с фичами

In [ ]:
dff_7 = dff_6.copy()
dff_7["split_reviews"] = dff_7["reviews"].apply(split_review)

In [ ]:
dff_7 = dff_7[dff_7["split_reviews"].apply(lambda x: len(x) > 0)]
dff_7 = dff_7.explode("split_reviews").reset_index(drop=True)

In [ ]:
dff_7 = dff_7[(dff_7["rating"] > 3) & (dff_7["rating_decimal"] > 3)]

In [ ]:
dff_7.dropna(
    subset=["split_reviews"],
    inplace=True,
)

In [ ]:
dff_7.drop_duplicates(
    subset=[
        "split_reviews",
    ],
    inplace=True,
)

In [ ]:
dff_7.drop(
    columns=[
        "reviews",
        "features",
        "reviews_clean",
        "features_clean",
        "reviews_clean_no_verbs",
        "features_clean_no_verbs",
        "reviews_clean_no_adj",
        "features_clean_no_adj",
        "reviews_clean_no_verbs_no_adj",
    ],
    inplace=True,
)

In [173]:
dff_7.to_csv("review_flamp_yandex_v7_keys_exploaded.csv", index=False)